# Artificial Neural Network

# Problem Description:

The bank has observed an unusually high churn rate, where a significant number of customers are leaving the bank. To address this issue, we have access to a dataset containing information about 10,000 customers over a six-month period, including whether or not they left the bank during that time. Our goal is to develop a predictive model that can identify customers at a higher risk of leaving the bank. By doing so, the bank can take proactive measures to retain these customers and reduce the churn rate.

### Importing the libraries

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
tf.__version__

'2.13.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [4]:
df = pd.read_csv('/content/Churn_Modelling.csv')
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [5]:
# rownumber, customerID and surname does not help with problem removing un-necessary columns
# taking x starting with index 3 which is redirt score
x = df.iloc[:, 3:-1].values
y = df.iloc[:, -1].values

In [6]:
print(x)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [7]:
print(y)

[1 0 1 ... 1 1 0]


### Encoding categorical data

Label Encoding the "Gender" column

In [8]:
# lable encoder for gender in x which is at index 2

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
x[:, 2] = le.fit_transform(x[:, 2])
print(x)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column

In [11]:
# one hot encode for converting str to int (o's and 1's)

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# below we add [1] because we want to apply onehot encoding on contry colomn whose index is [1]
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
x = np.array(ct.fit_transform(x))   # after onehotencoding dummy columns are moved to start

In [12]:
print(x)

[[1.0 0.0 1.0 ... 1 1 101348.88]
 [1.0 0.0 0.0 ... 0 1 112542.58]
 [1.0 0.0 1.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 1.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 1.0 ... 1 0 38190.78]]


In [32]:
x[0]

array([1.0, 0.0, 1.0, 0.0, 619, 0, 42, 2, 0.0, 1, 1, 1, 101348.88],
      dtype=object)

### Splitting the dataset into the Training set and Test set

In [13]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state = 0)

### Feature Scaling

In [14]:
from sklearn.preprocessing import StandardScaler  # feature scaling is imp for deeplearning

# apply feature scaling to only numerical values not encoded values as it will change encoded values
# we are appliying feature scaling to all the data for deeplearning
sc = StandardScaler()
x_train = sc.fit_transform(x_train)

# here we used only transform method as we dont want to create new scaler for train method it will use same scaler as train method to tramnsform
x_test = sc.transform(x_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [17]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [18]:
ann.add(tf.keras.layers.Dense(units= 6, activation='relu'))

### Adding the second hidden layer

In [19]:
ann.add(tf.keras.layers.Dense(units= 6, activation='relu'))

### Adding the output layer

In [20]:
ann.add(tf.keras.layers.Dense(units= 1, activation='sigmoid'))
# the final output using sigmoid function would be probability in range of (0,1), we wont be getting ans in 0 & 1

## Part 3 - Training the ANN

### Compiling the ANN

In [22]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # here we are using binary-cossentopy as we have only 2 class in output in case of 3 or more class we have to use 'categorical_crossentropy'

### Training the ANN on the Training set

In [23]:
ann.fit(x_train, y_train, epochs=100, batch_size=32) # so in bastchsize we are comparing bathch of 32 row to the actual value of y

Epoch 1/100
250/250 [==============================] - 2s 2ms/step - loss: 0.5680 - accuracy: 0.7206
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4574 - accuracy: 0.7954
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4366 - accuracy: 0.7981
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4261 - accuracy: 0.8073
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4170 - accuracy: 0.8200
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4094 - accuracy: 0.8242
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4028 - accuracy: 0.8282
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3978 - accuracy: 0.8284
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3936 - accuracy: 0.8305
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3899 - accura

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation



Use our ANN model to predict if the customer with the following informations will leave the bank:

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer ?

**Solution**

In [40]:
# input accepted by predict method is 2d array
# Create a list containing the input data for prediction
input_data = [[1, 0, 1, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]

# Scale the input data using the same scaler used during model training
scaled_input_data = sc.transform(input_data)

# Make the prediction
prediction = (ann.predict(scaled_input_data)  > 0.5) # to convert prediction range(0,1) in to 0's and 1's we can add int before ()

# Print the prediction : does cx leaves the bank? our last colom is exits means 1[True] if cx leave, 0[False] if cx stays
print(prediction)

1/1 [==============================] - 0s 31ms/step
[[False]]


Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

In [43]:
y_pred = (ann.predict(x_test) > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1)) # here at last 1 is for

63/63 [==============================] - 0s 1ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [44]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1487  108]
 [ 186  219]]


0.853